<a href="https://colab.research.google.com/github/sanjana1976/-hilde-dynamic-constraints/blob/master/hilde_constraint_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip uninstall torch torchvision torchaudio -y
#!pip cache purge
#!apt-get update
#!apt-get install -y python3-pip
!pip install --upgrade pip
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers>=4.30.0 openai>=1.0.0 numpy>=1.24.0

In [ ]:
import torch
import torchvision
import torchaudio
import transformers
import openai
import numpy as np

print(f"Torch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"Torchaudio version: {torchaudio.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"OpenAI version: {openai.__version__}")
print(f"Numpy version: {np.__version__}")

print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name() if torch.cuda.is_available() else "None")


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import os

# Ask the user for their API key
api_key = input("Please enter your OpenAI API key: ")

# Store it as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optional: confirm it's set (don't print the key itself for safety)
if os.environ.get("OPENAI_API_KEY"):
    print("API key set successfully!")
else:
    print("API key not set.")

In [ ]:

# HiLDe Completion Engine

# Simplified code generation using Qwen2.5-Coder-7B


import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import warnings
warnings.filterwarnings("ignore")


class HiLDeLiteCompletionEngine:
    def __init__(self, model_name="Qwen/Qwen2.5-Coder-7B-Instruct"):
        #Initialize the completion engine with Qwen2.5-Coder-7B
        print(f"Loading completion model: {model_name}")

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True
        )

        # Load model with optimizations for Colab
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True
        )

        print("Completion engine loaded successfully!")

    def generate_completion(self, prompt, max_tokens=100, temperature=0.7):

       # Generate a single code completion
       # Arguments:
          #  prompt (str): The input prompt
          #  max_tokens (int): Maximum tokens to generate
          # temperature (float): Sampling temperature
        #Returns:
            #str: Generated code completion

        try:
            # Tokenize input
            inputs = self.tokenizer.encode(prompt, return_tensors="pt")
            inputs = inputs.to(self.model.device)

            # Generate completion
            with torch.no_grad():
                outputs = self.model.generate(
                    inputs,
                    max_new_tokens=max_tokens,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            # Decode output
            completion = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Remove the original prompt from completion
            if completion.startswith(prompt):
                completion = completion[len(prompt):].strip()

            return completion

        except Exception as e:
            print(f"Error generating completion: {e}")
            return "# Error: Could not generate completion"


if __name__ == "__main__":
    # Test the completion engine
    engine = HiLDeLiteCompletionEngine()

    test_prompt = "def hash_password(password):"
    print(f"Prompt: {test_prompt}")

    completion = engine.generate_completion(test_prompt, max_tokens=50)
    print(f"Generated: {completion}")


In [ ]:

#Dynamic Rule Translator

#Converts natural language constraints to complete rule specifications using LLM.
#This replaces the hardcoded dictionary approach with  dynamic rule generation.


import os
import json
import openai
from typing import List, Dict, Any, Optional


class DynamicRuleTranslator:
    def __init__(self, api_key: Optional[str] = None):
        #Initialize the rule translator
        if not api_key:
            api_key = os.getenv("OPENAI_API_KEY")

        if api_key:
            openai.api_key = api_key
            self.api_key = api_key
        else:
            print("Warning: No API key provided. Using fallback translation.")
            self.api_key = None

    def translate_constraints(self, user_constraints: List[str])-> List[Dict[str, Any]]:

       # Translate natural language constraints to complete rule specifications

        #Args:
          #user_constraints: List of natural language constraint descriptions
        # Returns:
         #  List of complete rule dictionaries with all necessary information

        translated_rules = []

        for constraint in user_constraints:
            rule_spec = self._translate_single_constraint(constraint)
            if rule_spec:
                translated_rules.append(rule_spec)

        return translated_rules

    def _translate_single_constraint(self, constraint: str) :
        #Translate a single constraint using LLM
        if self.api_key:
            try:
                return self._llm_translate_constraint(constraint)
            except Exception as e:
                print(f"LLM translation error: {e}")
                return self._fallback_translate(constraint)
        else:
            return self._fallback_translate(constraint)

    def _llm_translate_constraint(self, constraint: str) -> Dict[str, Any]:

       # Use LLM to generate complete rule specification from natural language

        prompt = f"""You are a code analysis expert. Convert this natural language constraint into a complete rule specification.

User constraint: "{constraint}"

Return a JSON object with the following structure:
{{
    "rule_name": "snake_case_rule_name",
    "ast_node_type": "ASTNodeType",
    "message": "Human readable violation message",
    "severity": "error|warning|info",
    "description": "Brief description of what this rule checks",
    "additional_checks": {{
        "function_name": "optional_function_name",
        "module_name": "optional_module_name",
        "attribute_name": "optional_attribute_name"
    }}
}}

Rules for translation:
1. rule_name: Use snake_case, be specific and technical
2. ast_node_type: Use the exact Python AST node type name (e.g., "While", "For", "Call", "Import", etc.)
3. message: Clear, actionable message for developers
4. severity: "error" for critical violations, "warning" for style issues, "info" for suggestions
5. description: Brief explanation of what the rule checks
6. additional_checks: Optional object for specific checks

Common patterns:
- "no while loops" → ast_node_type: "While"
- "no function calls" → ast_node_type: "Call"
- "no global variables" → ast_node_type: "Assign" (at module level)
- "no imports" → ast_node_type: "Import"
- "no classes" → ast_node_type: "ClassDef"
- "no eval" → ast_node_type: "Call", additional_checks.function_name: "eval"
- "no requests library" → ast_node_type: "Import", additional_checks.module_name: "requests"

Return ONLY the JSON object, no other text or explanation.
"""

        try:
            response = openai.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a code analysis expert. Convert natural language constraints to complete technical rule specifications."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=300,
                temperature=0.1,
            )

            result_text = response.choices[0].message.content.strip()

            # Clean up markdown (if LLM adds it)
            if result_text.startswith("```json"):
                result_text = result_text.removeprefix("```json").strip()
            if result_text.endswith("```"):
                result_text = result_text.removesuffix("```").strip()

            rule_spec = json.loads(result_text)

            required_fields = ["rule_name", "ast_node_type", "message", "severity"]
            for field in required_fields:
                if field not in rule_spec:
                    raise ValueError(f"Missing required field: {field}")

            if "additional_checks" not in rule_spec:
                rule_spec["additional_checks"] = {}

            return rule_spec

        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}")
            print(f"LLM response: {result_text}")
            return self._fallback_translate(constraint)
        except Exception as e:
            print(f"LLM translation error: {e}")
            return self._fallback_translate(constraint)

    def _fallback_translate(self, constraint: str) -> Dict[str, Any]:
        #Fallback translation when LLM is unavailable
        constraint_lower = constraint.lower().strip()

        fallback_rules = {
            "while": {"ast_node_type": "While", "message": "While loops are not allowed", "severity": "error"},
            "for": {"ast_node_type": "For", "message": "For loops are not allowed", "severity": "error"},
            "function call": {"ast_node_type": "Call", "message": "Function calls are not allowed", "severity": "error"},
            "global": {"ast_node_type": "Assign", "message": "Global variables are not allowed", "severity": "warning"},
            "import": {"ast_node_type": "Import", "message": "Import statements are not allowed", "severity": "error"},
            "class": {"ast_node_type": "ClassDef", "message": "Class definitions are not allowed", "severity": "error"},
            "recursion": {"ast_node_type": "Call", "message": "Recursive calls are not allowed", "severity": "error"},
            "lambda": {"ast_node_type": "Lambda", "message": "Lambda functions are not allowed", "severity": "warning"},
            "eval": {"ast_node_type": "Call", "message": "Eval function is not allowed", "severity": "error"},
        }

        for keyword, rule_info in fallback_rules.items():
            if keyword in constraint_lower:
                rule_name = constraint.replace(" ", "_").replace("-", "_").lower()
                return {
                    "rule_name": rule_name,
                    "ast_node_type": rule_info["ast_node_type"],
                    "message": rule_info["message"],
                    "severity": rule_info["severity"],
                    "description": f"Fallback rule for: {constraint}",
                    "additional_checks": {}
                }

        # Default fallback
        rule_name = constraint.replace(" ", "_").replace("-", "_").lower()
        return {
            "rule_name": rule_name,
            "ast_node_type": "Call",
            "message": f"Violation of constraint: {constraint}",
            "severity": "warning",
            "description": f"Fallback rule for: {constraint}",
            "additional_checks": {}
        }


if __name__ == "__main__":
    # Test the dynamic rule translator
    translator = DynamicRuleTranslator()

    test_constraints = [
        "don't allow while loops",
        "no function calls",
        "ban global variables",
        "no imports please",
        "don't use eval function",
        "no requests library"
    ]

    print("Testing Dynamic Rule Translator")
    print("=" * 50)

    for constraint in test_constraints:
        print(f"\nConstraint: '{constraint}'")
        rule_spec = translator._translate_single_constraint(constraint)
        print(f"Generated Rule:")
        print(json.dumps(rule_spec, indent=2))


In [ ]:
# Generic Dynamic AST Checker
#
# Finds violations in Python code based on dynamic rule specifications.
# This is now truly generic - no hardcoded rule mappings required!

import ast
from typing import List, Dict, Any, Optional


def find_violations_in_ast(source_code: str, rule_specifications: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    # Traverse AST to find violations based on dynamic rule specifications
    #
    # Args:
    #     source_code: Python code to analyze
    #     rule_specifications: List of complete rule dictionaries from DynamicRuleTranslator
    #
    # Returns:
    #     List of violation dictionaries
    violations = []

    try:
        tree = ast.parse(source_code)
    except SyntaxError as e:
        return [{
            "rule_name": "syntax_error",
            "line": e.lineno or 0,
            "column": e.offset or 0,
            "message": f"Syntax error: {e.msg}",
            "severity": "error",
            "code_snippet": "",
            "description": "Python syntax error in the code"
        }]

    # Create visitor to check rule specifications
    visitor = DynamicConstraintVisitor(rule_specifications, source_code)
    visitor.visit(tree)

    return visitor.violations


class DynamicConstraintVisitor(ast.NodeVisitor):
    # Generic AST visitor that checks for violations based on dynamic rule specifications

    def __init__(self, rule_specifications: List[Dict[str, Any]], source_code: str):
        self.rule_specifications = rule_specifications
        self.source_code = source_code
        self.source_lines = source_code.split('\n')
        self.violations = []
        self.current_line = 0
        self.function_names = set()  # Track function names for recursion detection
        self.current_function = None  # Track current function for recursion detection

    def visit(self, node):
        # Visit each node and check all rule specifications
        self.current_line = getattr(node, 'lineno', 0)

        # Track function definitions for recursion detection
        if isinstance(node, ast.FunctionDef):
            self.function_names.add(node.name)
            self.current_function = node.name

        # Check each rule specification against this node
        for rule_spec in self.rule_specifications:
            if self._check_rule_violation(node, rule_spec):
                violation = self._create_violation(node, rule_spec)
                self.violations.append(violation)

        # Continue visiting child nodes
        self.generic_visit(node)

    def _check_rule_violation(self, node: ast.AST, rule_spec: Dict[str, Any]) -> bool:
        # Check if a node violates a specific rule specification
        #
        # This is the key improvement: we now read the ast_node_type directly
        # from the rule specification instead of using hardcoded mappings!
        expected_node_type = rule_spec.get('ast_node_type', '')
        rule_name = rule_spec.get('rule_name', '')
        additional_checks = rule_spec.get('additional_checks', {})

        # Check if current node matches the expected AST node type
        if not expected_node_type or type(node).__name__ != expected_node_type:
            return False

        # Perform additional checks if specified
        if additional_checks:
            return self._perform_additional_checks(node, rule_spec, additional_checks)

        # For rules that need special logic beyond just node type matching
        if rule_name.endswith('_global_vars'):
            return self._is_global_assignment(node)
        elif rule_name.endswith('_recursion'):
            return self._is_recursive_call(node)
        elif rule_name.endswith('_function_calls'):
            return True  # Any function call is a violation
        else:
            return True  # Node type matches, so it's a violation

    def _perform_additional_checks(self, node: ast.AST, rule_spec: Dict[str, Any], additional_checks: Dict[str, Any]) -> bool:
        # Perform additional checks based on the rule specification

        # Check for specific function names (e.g., "no eval")
        if 'function_name' in additional_checks:
            expected_function = additional_checks['function_name']
            if isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
                return node.func.id == expected_function

        # Check for specific module names (e.g., "no requests library")
        if 'module_name' in additional_checks:
            expected_module = additional_checks['module_name']
            if isinstance(node, ast.Import):
                for alias in node.names:
                    if alias.name == expected_module:
                        return True
            elif isinstance(node, ast.ImportFrom):
                if node.module == expected_module:
                    return True

        # Check for specific attribute names
        if 'attribute_name' in additional_checks:
            expected_attribute = additional_checks['attribute_name']
            if isinstance(node, ast.Attribute):
                return node.attr == expected_attribute

        # If no specific additional checks match, return True (node type violation)
        return True

    def _is_global_assignment(self, node: ast.Assign) -> bool:
        # Check if assignment is at module level (global)
        if not isinstance(node, ast.Assign):
            return False

        # Simple heuristic: if we're at the top level of the AST
        # In a more sophisticated implementation, you'd track the AST depth
        return True

    def _is_recursive_call(self, node: ast.Call) -> bool:
        # Check if function call is recursive
        if not isinstance(node, ast.Call) or not isinstance(node.func, ast.Name):
            return False

        # Check if the function being called is the same as the current function
        if self.current_function and node.func.id == self.current_function:
            return True

        # Check if the function being called is in our known function names
        # (This is a simple heuristic - a more sophisticated approach would
        # analyze the actual call graph)
        if node.func.id in self.function_names:
            return True

        return False

    def _create_violation(self, node: ast.AST, rule_spec: Dict[str, Any]) -> Dict[str, Any]:
        # Create a violation dictionary from the rule specification
        rule_name = rule_spec.get('rule_name', 'unknown_rule')
        message = rule_spec.get('message', f'Violation of {rule_name}')
        severity = rule_spec.get('severity', 'warning')
        description = rule_spec.get('description', '')

        # Get code snippet
        code_snippet = ""
        if hasattr(node, 'lineno') and node.lineno:
            line_idx = node.lineno - 1
            if 0 <= line_idx < len(self.source_lines):
                code_snippet = self.source_lines[line_idx].strip()

        return {
            "rule_name": rule_name,
            "line": getattr(node, 'lineno', 0),
            "column": getattr(node, 'col_offset', 0),
            "message": message,
            "severity": severity,
            "code_snippet": code_snippet,
            "description": description,
            "ast_node_type": type(node).__name__
        }


def get_available_ast_node_types() -> List[str]:
    # Get list of available Python AST node types for reference
    return [
        "While", "For", "Call", "Import", "ImportFrom", "Assign", "ClassDef",
        "FunctionDef", "Lambda", "ListComp", "DictComp", "GeneratorExp",
        "IfExp", "Assert", "Raise", "Try", "With", "AsyncFunctionDef",
        "AsyncFor", "AsyncWith", "Name", "Attribute", "BinOp", "UnaryOp",
        "Compare", "BoolOp", "If", "Return", "Yield", "YieldFrom", "Delete",
        "AugAssign", "Global", "Nonlocal", "Pass", "Break", "Continue"
    ]


if __name__ == "__main__":
    # Test the dynamic AST checker
    test_code = """
def test_function():
    import os
    global_var = "test"

    while True:
        result = calculate(5)
        if result:
            break

    eval("print('hello')")
    return result
"""

    # Example rule specifications (what the DynamicRuleTranslator would generate)
    test_rule_specs = [
        {
            "rule_name": "no_while_loops",
            "ast_node_type": "While",
            "message": "While loops are not allowed",
            "severity": "error",
            "description": "Checks for while loop usage",
            "additional_checks": {}
        },
        {
            "rule_name": "no_global_vars",
            "ast_node_type": "Assign",
            "message": "Global variables are not allowed",
            "severity": "warning",
            "description": "Checks for global variable assignments",
            "additional_checks": {}
        },
        {
            "rule_name": "no_imports",
            "ast_node_type": "Import",
            "message": "Import statements are not allowed",
            "severity": "error",
            "description": "Checks for import statements",
            "additional_checks": {}
        },
        {
            "rule_name": "no_eval_function",
            "ast_node_type": "Call",
            "message": "Eval function is not allowed",
            "severity": "error",
            "description": "Checks for eval function usage",
            "additional_checks": {
                "function_name": "eval"
            }
        }
    ]

    print("Testing Dynamic AST Checker")
    print("=" * 50)
    print("Test code:")
    print(test_code)
    print("\nRule specifications:")
    for spec in test_rule_specs:
        print(f"- {spec['rule_name']}: {spec['ast_node_type']}")

    violations = find_violations_in_ast(test_code, test_rule_specs)

    print(f"\nViolations found: {len(violations)}")
    for violation in violations:
        print(f"- {violation['rule_name']} (Line {violation['line']}, {violation['severity']}): {violation['message']}")
        if violation['code_snippet']:
            print(f"  Code: {violation['code_snippet']}")


In [ ]:

# HiLDe Analysis Engine

# GPT-4 powered summarization of constraint violations, API KEY NEEDED FOR THIS PART


import os
import json
from typing import List, Dict, Any
from openai import OpenAI


class GPT4AnalysisEngine:
    def __init__(self, api_key=None):
        # Initialize the GPT-4 analysis engine
        if not api_key:
            api_key = os.getenv("OPENAI_API_KEY")

        if api_key:
            self.client = OpenAI(api_key=api_key)
            self.api_key = api_key
        else:
          #Fall back plan
            print("Warning: No API key provided. Using fallback summaries.")
            self.client = None
            self.api_key = None

    def summarize_violations(self, violations: List[Dict[str, Any]]) :

       # Summarize constraint violations using GPT-4
       # Args:
           # violations: List of violation dictionaries
      # Returns:
           # str: Human-readable summary

        if not violations:
            return "No constraint violations found. Code looks good!"

        try:
            if self.client:
                return self._gpt4_summary(violations)
            else:
                return self._create_fallback_summary(violations)
        except Exception as e:
            print(f"Error in GPT-4 analysis: {e}")
            return self._create_fallback_summary(violations)

    def _gpt4_summary(self, violations: List[Dict[str, Any]]):
        """Generate summary using GPT-4 API"""
        violations_json = json.dumps(violations, indent=2)

        prompt = f"""Act as a senior software engineer and code quality expert. The following JSON contains constraint violations found in a Python code snippet:

{violations_json}

Please provide a concise, high-level summary that:
1. Explains what problems were found
2. Suggests how to fix each issue
3. Uses clear, non-technical language suitable for developers
4. Focuses on the most critical issues first

Format your response as a clear, actionable summary. Do not include a closing or sign-off like "Best" or a name. be direct and formal"""

        try:
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a senior software engineer and code quality expert."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=500,
                temperature=0.3
            )

            return response.choices[0].message.content.strip()

        except Exception as e:
            print(f"GPT-4 API error: {e}")
            return self._create_fallback_summary(violations)

    def _create_fallback_summary(self, violations: List[Dict[str, Any]]) :
        # basic summary when GPT-4 is unavailable
        if not violations:
            return "No violations found."

        summary_parts = [f"Found {len(violations)} constraint violation(s):"]

        for i, violation in enumerate(violations, 1):
            rule = violation.get('rule_name', 'Unknown rule')
            line = violation.get('line', 'Unknown line')
            message = violation.get('message', 'No message')
            severity = violation.get('severity', 'info')

            summary_parts.append(f"{i}. **{rule}** (Line {line}, {severity}): {message}")

        return "\n".join(summary_parts)


if __name__ == "__main__":
    # Test the analysis engine
    engine = GPT4AnalysisEngine()

    test_violations = [
        {
            "rule_name": "no_while_loops",
            "line": 5,
            "message": "While loops are not allowed",
            "severity": "error"
        },
        {
            "rule_name": "no_global_vars",
            "line": 3,
            "message": "Global variables are not allowed",
            "severity": "warning"
        }
    ]

    summary = engine.summarize_violations(test_violations)
    print("Testing....Summary:")
    print(summary)


In [ ]:

# Dynamic Constraint System

# Integrates completion, dynamic rule translation, and generic AST checking.

from typing import List, Dict, Any
import json


class DynamicConstraintSystem:
    def __init__(self, completion_engine, analysis_engine, api_key=None):
        self.completion_engine = completion_engine
        self.analysis_engine = analysis_engine
        self.rule_translator = DynamicRuleTranslator(api_key)

        print("Dynamic Constraint System initialized.")

    def process_code_with_constraints(self, prompt: str, user_constraints: List[str], max_tokens: int = 100) -> Dict[str, Any]:
        print("Starting processing:")

        print("Generating code:")
        completion = self.completion_engine.generate_completion(prompt, max_tokens)

        print("Translating constraints:")
        rule_specifications = self.rule_translator.translate_constraints(user_constraints)

        print("Checking code:")
        violations = find_violations_in_ast(completion, rule_specifications)

        print("Summarizing results:")
        summary = self.analysis_engine.summarize_violations(violations)

        return {
            "completion": completion,
            "user_constraints": user_constraints,
            "rule_specifications": rule_specifications,
            "violations": violations,
            "summary": summary,
            "violation_count": len(violations),
            "constraints_checked": [spec['rule_name'] for spec in rule_specifications]
        }

    def add_custom_constraint(self, constraint_description: str)-> Dict[str, Any]:
        print(f"Adding constraint: '{constraint_description}'")

        rule_specs = self.rule_translator.translate_constraints([constraint_description])
        if rule_specs:
            rule_spec = rule_specs[0]
            print(f"Rule: {rule_spec['rule_name']} ({rule_spec['ast_node_type']})")
            return rule_spec
        else:
            print("Failed to add constraint.")
            return {}

    def test_constraint_on_code(self, code: str, constraint_description: str) -> Dict[str, Any]:
        print(f"Testing constraint '{constraint_description}' on code...")

        rule_specs = self.rule_translator.translate_constraints([constraint_description])
        if not rule_specs:
            return {"error": "Failed to translate constraint"}

        violations = find_violations_in_ast(code, rule_specs)
        summary = self.analysis_engine.summarize_violations(violations)

        return {
            "constraint": constraint_description,
            "rule_specification": rule_specs[0],
            "violations": violations,
            "summary": summary,
            "violation_count": len(violations)
        }

    def get_system_info(self) -> Dict[str, Any]:
        return {
            "system_type": "Dynamic Constraint System",
            "features": [
                "Dynamic rule translation using LLM",
                "Generic AST checking (no hardcoded rules)",
                "Extensible constraint system",
                "Natural language constraint input",
                "Intelligent violation summarization"
            ],
            "capabilities": [
                "Add new constraints without code changes",
                "Support for any Python AST node type",
                "Additional checks for specific functions/modules",
                "Configurable severity levels",
                "Human-readable violation messages"
            ],
            "example_constraints": [
                "don't allow while loops",
                "no function calls",
                "ban global variables",
                "no imports",
                "don't use eval function",
                "no requests library",
                "no database connections",
                "no file operations",
                "no network requests",
                "no recursion"
            ]
        }

    def demonstrate_extensibility(self):
        print("Demonstrating dynamic system extensibility")
        print("===============================================================")

        new_constraints = [
            "no pandas library usage",
            "don't allow database connections",
            "ban file operations",
            "no network requests",
            "no threading",
            "no multiprocessing",
            "no subprocess calls",
            "no os.system calls",
            "no eval or exec",
            "no pickle operations"
        ]

        print("Adding new constraints:")
        for constraint in new_constraints:
            rule_spec = self.add_custom_constraint(constraint)
            if rule_spec:
                print(f"'{constraint}' → {rule_spec['ast_node_type']} ({rule_spec['severity']})")
            else:
                print(f"Failed to process: '{constraint}'")

        print("\nAll constraints added without code changes.")


if __name__ == "__main__":
    print("Testing Dynamic Constraint System")
    print("===================================================================")

    class MockCompletionEngine:
        def generate_completion(self, prompt, max_tokens):
            return f"""def {prompt.split('(')[0].split()[-1] if '(' in prompt else 'test_function'}():
    import os
    global_var = "test"

    while True:
        result = calculate(5)
        if result:
            break

    eval("print('hello')")
    return result"""

    class MockAnalysisEngine:
        def summarize_violations(self, violations):
            if not violations:
                return "No violations found."

            summary_parts = [f"Found {len(violations)} violation(s):"]
            for i, violation in enumerate(violations, 1):
                summary_parts.append(f"{i}. {violation['rule_name']} (Line {violation['line']}, {violation['severity']}): {violation['message']}")
            return "\n".join(summary_parts)

    completion_engine = MockCompletionEngine()
    analysis_engine = MockAnalysisEngine()
    constraint_system = DynamicConstraintSystem(completion_engine, analysis_engine)

    prompt = "def process_data():"
    user_constraints = [
        "don't allow while loops",
        "no function calls",
        "ban global variables",
        "no imports",
        "don't use eval function"
    ]

    result = constraint_system.process_code_with_constraints(prompt, user_constraints)

    print("=== RESULT ===")
    print(f"Generated Code:\n{result['completion']}\n")
    print(f"User Constraints: {result['user_constraints']}")
    print(f"Rules Generated: {len(result['rule_specifications'])}")
    for spec in result['rule_specifications']:
        print(f" - {spec['rule_name']}: {spec['ast_node_type']}")
    print(f"Violations Found: {result['violation_count']}")
    print("Summary:")
    print(result['summary'])

    print("\n" + "============================================================")
    constraint_system.demonstrate_extensibility()
